In [1]:
import openai
import os

import requests
from bs4 import BeautifulSoup

from PIL import Image

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [3]:
url = input("Enter URL for recipe: ") 

In [4]:
#url = 'https://thehiddenveggies.com/silken-tofu-pasta-sauce/'
text = f"""
Visit the website at the following URL: {url}
"""
prompt = f"""
From the text below, please understand it as an article that describes a recipe.  
The first line should be: "<recipe name>". Next comes the ingredient list in the form of "Ingredients:" followed by 
each line should contain one ingredient and it's amount in the format
"<ingredient>: <amount>". The ingredient part (before the semicolon) should contain only the ingredient name, not the amount.
Next is the "Instructions:" section.  Each line should be a numbered list for each step.  And lastly, there may be a notes section. 
If there is it should be titled "Notes:" with a paragraph after.
      The text is: 
```{text}```
"""
response = get_completion(prompt)
print(response)

Pumpkin Spice Protein Bars Recipe
Ingredients:
- 1 cup pumpkin puree
- 1/2 cup almond butter
- 1/4 cup maple syrup
- 2 scoops vanilla protein powder
- 1/2 cup almond flour
- 1/4 cup ground flaxseed
- 1 teaspoon pumpkin pie spice
- 1/2 teaspoon cinnamon
- 1/4 teaspoon salt
- 1/2 cup dark chocolate chips

Instructions:
1. Preheat the oven to 350°F (175°C) and line a baking dish with parchment paper.
2. In a large mixing bowl, combine the pumpkin puree, almond butter, and maple syrup. Mix well until smooth.
3. Add the vanilla protein powder, almond flour, ground flaxseed, pumpkin pie spice, cinnamon, and salt to the bowl. Stir until all the ingredients are well combined.
4. Fold in the dark chocolate chips.
5. Pour the mixture into the prepared baking dish and spread it evenly.
6. Bake in the preheated oven for 25-30 minutes, or until the edges are golden brown and a toothpick inserted into the center comes out clean.
7. Remove from the oven and let it cool completely before cutting into 

In [ ]:

with open(os.path.join("/Users/melsmac/Desktop/Portfolio Proj/Cookbook/PrintedRecipes",response.split('\n')[0]+".md"), "w") as file:
    file.write(response)

In [28]:
import requests
from PIL import Image
from io import BytesIO
import openai

def generate(text):
    res = openai.Image.create(
        prompt=text,
        n=1,
        size="256x256",
    )
    # Check if the response contains a valid image URL
    if "data" in res and res["data"] and "url" in res["data"][0]:
        return res["data"][0]["url"]
    else:
        return None

# Call the custom function "generate"
url1 = generate(response)

if url1:
    # Use requests library to get the image in bytes
    response = requests.get(url1)
    try:
        # Use the Image module from PIL library to view the image
        img = Image.open(BytesIO(response.content))
        img.show()
    except UnidentifiedImageError as e:
        print(f"Error: Unable to open the image. The URL might not contain a valid image.")
else:
    print("Error: The API response did not contain a valid image URL.")
